# Analyze Zarr Image from a public S3 repository using ilastik 
The notebook shows how to load an IDR image converted into a Zarr file.

The image is then analyzed using [ilastik](https://ilastik.github.io/).
The image is taken from  the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388

The images can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=project-801).

Binary data are read from a public S3 repository.
The dimension might need to be adjusted depending on the ilastik project.

### Insert required packages

In [1]:
import os
import numpy
import zarr
import dask.array as da
from dask.diagnostics import ProgressBar

from collections import OrderedDict

from ilastik import app
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo

# package for 3d visualization
from itkwidgets import compare, view

The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


### Enter the image ID

In [2]:
image_id = 6001240

### Load an Image as 5D-numpy array from S3

In [3]:
def load_from_s3(id, resolution='0'):
    endpoint_url = 'https://s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    # data.shape is (t, c, z, y, x) by convention
    print(da.from_zarr(endpoint_url + root))
    with ProgressBar():
        return numpy.asarray(da.from_zarr(endpoint_url + root))

### Load data

In [4]:
%time input_data = load_from_s3(image_id)

dask.array<from-zarr, shape=(1, 2, 236, 275, 271), dtype=>u2, chunksize=(1, 1, 1, 275, 271), chunktype=numpy.ndarray>
[########################################] | 100% Completed | 18.5s
CPU times: user 3.8 s, sys: 1.72 s, total: 5.53 s
Wall time: 18.9 s


### Store the channels data so we can compare them with the results of the pixel classification later.

In [5]:
first_channel = input_data[0, 0, :, : ,:]
second_channel = input_data[0, 1, :, : ,:]

### Load each image as a 5D-numpy array and analyze.

In [6]:
# Load the model linked to the dataset
home = os.path.expanduser("~")
model_file = home+"/notebooks/pipelines/pixel-class-133.ilp"
# Re-order the array tczyx -> tzyxc
input_data = input_data.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
print(input_data.shape)
# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = app.parse_args([])
args.headless = True
args.project = model_file
shell = app.main(args)

print('running ilastik using %s' % model_file)
role_data_dict = OrderedDict(
[
    (
        "Raw Data",
        [
            PreloadedArrayDatasetInfo(preloaded_array=input_data)
        ],
    )
])
predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
for data in predictions:
    # Re-organise array from tzyxc to tczyx order
    data = data.swapaxes(4, 3).swapaxes(3, 2).swapaxes(2, 1)
print("done")

(1, 236, 275, 271, 2)
INFO ilastik.app: Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".
Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".
INFO ilastik.app: Resetting lazyflow thread pool with 2 threads.
INFO ilastik.app: Configuring lazyflow RAM limit to 2.0GiB
INFO lazyflow.utility.memory: Available memory set to 2.0GiB


WARNING 2020-07-10 09:10:59,164 memory 110 140051834550080 User specified memory exceeds memory physically available. Please check theconfiguration.
WARNING 2020-07-10 09:11:00,331 __init__ 110 140051834550080 Failed to automatic tracking workflow; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory
WARNING 2020-07-10 09:11:00,336 __init__ 110 140051834550080 Failed to import automatic tracking workflow (conservation tracking). For this workflow, see the installationinstructions on our website ilastik.org; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory
WARNING 2020-07-10 09:11:00,356 __init__ 110 140051834550080 Failed to import structured learning tracking workflow. For this workflow, see the installationinstructions on our website ilastik.org; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory


INFO ilastik.shell.projectManager: Opening Project: /home/jmarie/notebooks/pipelines/pixel-class-133.ilp


WARNING opSimpleBlockedArrayCache.py(43): FutureWarning: Conversion of the second argument of issubdtype from `dtype` to `np.generic` is deprecated. In future, it will be treated as `np.object_ == np.dtype(dtype).type`.
WARNING opSlicedBlockedArrayCache.py(141): FutureWarning: Conversion of the second argument of issubdtype from `dtype` to `np.generic` is deprecated. In future, it will be treated as `np.object_ == np.dtype(dtype).type`.


running ilastik using /home/jmarie/notebooks/pipelines/pixel-class-133.ilp
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to in-memory array.
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 504.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 1.5GiB, split between 2 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (1, 92, 92, 92, 2)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 748.6MiB


WARNING arraytypes.py(1263): FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
WARNING arraytypes.py(1269): FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


DEBUG lazyflow.operators.classifierOperators: Features took 5.478854 seconds. Prediction took 2.139839 seconds. Subregion: start '[0, 0, 0, 0]' stop '[92, 92, 92, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 8.867785 seconds. Prediction took 2.017484 seconds. Subregion: start '[0, 0, 92, 0]' stop '[92, 92, 184, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 5.008051 seconds. Prediction took 2.820926 seconds. Subregion: start '[0, 0, 184, 0]' stop '[92, 92, 271, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 7.662044 seconds. Prediction took 3.075553 seconds. Subregion: start '[0, 92, 0, 0]' stop '[92, 184, 92, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 8.729481 seconds. Prediction took 1.903667 seconds. Subregion: start '[0, 92, 92, 0]' stop '[92, 184, 184, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 6.224377 seconds. Prediction took 2.003933 seconds. Subregion: start '[0, 92, 184, 0]' stop '[

### View the first channel and the analysis result side-by-side

In [11]:
compare(first_channel, data[0, 0, :, :, :], shadow=False, gradient_opacity=0.2, ui_collapsed=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

### Combine the  pixels classification map and the image.

In [10]:
names = [(0, 'First channel'), (1, 'Pixels Classification')]
viewer = view(first_channel,
              label_image=data[0, 0, :, :, :],
              label_image_names=names,
              label_image_blend=0.8,
              gradient_opacity=0.5,
              slicing_planes=False)
viewer

Viewer(geometries=[], interpolation=False, label_image_blend=0.9, label_image_names=[(0, 'First channel'), (1,…

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.